In [0]:
# First we import are liberiers
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
import pandas as pd
import mlflow
import mlflow.sklearn

In [0]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, f1_score
import mlflow
import mlflow.sklearn

# we first load the data 
df = pd.read_csv('/Workspace/Users/deepakpvinodsharma@gmail.com/THE_LEARNING_BRICK/Machine-learning-and-mlflow/spam.csv', 
                 encoding='ISO-8859-1', 
                 usecols=[0, 1])

# assign columns
df.columns = ['label', 'text']

df['label'] = df['label'].map({'ham': 0, 'spam': 1})

# Now we Split the data into train and test
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['label'], test_size=0.2, random_state=42)

# we vectorize the text data
vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# now we Train the model
model = MultinomialNB()

# Start the MLflow run
with mlflow.start_run():
    model = model.fit(X_train_tfidf, y_train)
    
    # Predict and evaluate
    y_pred = model.predict(X_test_tfidf)
    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    
    # now we are logging the vector metric
    mlflow.log_metric('accuracy', acc)
    mlflow.log_metric('f1_score', f1)
    
    # now we log the model
    mlflow.sklearn.log_model(model, "naive_bayes_model")
    
    # Saveing and logging the vec 
    import joblib
    joblib.dump(vectorizer, "/dbfs/tmp/vectorizer.pkl")
    mlflow.log_artifact("/dbfs/tmp/vectorizer.pkl")
    
    print(f"Accuracy: {acc}")
    print(f"F1 Score: {f1}")